In [ ]:
def create_degradation_df(df_degradation, df_eval, df_consistency):
    df_eval_for_degradation = df_eval[df_degradation.columns].copy()
    df_consistency_for_degradation = pd.concat([df_consistency]*len(df_eval), sort=True)
    df_consistency_for_degradation = df_consistency_for_degradation[df_degradation.columns].copy()
    df_consistency_for_degradation.index = df_eval_for_degradation.index

    df_consistency_for_degradation['type'] = 'H-H'
    df_eval_for_degradation['type'] = 'H-M'
    df_degradation['type'] = 'diff'

    df = pd.concat(
        [df_consistency_for_degradation,
         df_eval_for_degradation,
         df_degradation], sort=True)
    return df


if not out_dfs['consistency'].empty:

    markdown_strs = ['## Consistency']

    # human human agreement
    markdown_strs.append('### Human-human agreement')
    display(Markdown('\n'.join(markdown_strs)))
    
    display(HTML(out_dfs['consistency'].to_html(index=False,
                                                escape=False,
                                                float_format=float_format_func,
                                                classes=['alternate_colors2'])))

    # degradation
    display(Markdown('### Degradation'))
    df_eval_old = outputs_old['df_eval_for_degradation']
    df_eval_new = outputs_new['df_eval_for_degradation']
    df_degradation_old = outputs_old['df_degradation']
    df_degradation_new = outputs_new['df_degradation']
    df_consistency_old = outputs_old['df_consistency']
    df_consistency_new = outputs_new['df_consistency']

    # create placeholder dataframes if we have no data for consistency.
    # We already checked that we have data for at least one of the models,
    # so we can use the other model. 
    if df_consistency_old.empty:
        df_consistency_old = pd.DataFrame(columns=df_consistency_new.columns, 
                                          index=df_consistency_new.index,
                                          data='-')
        df_degradation_old = pd.DataFrame(columns=df_degradation_new.columns, 
                                          index=df_degradation_new.index,
                                          data='-')
        # use existing evaluation if exists else substitute with a placeholder
        if df_eval_old.empty:
            df_eval_old.empty =  pd.DataFrame(columns=df_eval_new.columns, 
                                              index=df_eval_new.index,
                                              data='-')
    if df_consistency_new.empty:
        df_consistency_new = pd.DataFrame(columns=df_consistency_old.columns, 
                                          index=df_consistency_old.index,
                                          data='-')
        df_degradation_new = pd.DataFrame(columns=df_degradation_old.columns, 
                                          index=df_degradation_old.index,
                                          data='-')
        if df_eval_new.empty:
            df_eval_new.empty =  pd.DataFrame(columns=df_eval_old.columns, 
                                              index=df_eval_old.index,
                                              data='-')
            
        
    #old
    df_degradation_old = create_degradation_df(df_degradation_old, df_eval_old, df_consistency_old)
    df_degradation_old['version'] = 'old'

    # new
    df_degradation_new = create_degradation_df(df_degradation_new, df_eval_new, df_consistency_new)
    df_degradation_new['version'] = 'new'
    
    
    df = pd.concat([df_degradation_old, df_degradation_new], sort=True)
    df = df[['type', 'version', 'corr', 'kappa',
             'wtkappa', 'exact_agr', 'adj_agr', 'SMD']]

    df.index.name = 'for'
    df = df.reset_index().sort_values(by=['for', 'version', 'type'],
                                      ascending=[True, False, True]).set_index(df.index.names)
    df.index.name = None

    # display the HTML for the table with the various formatters
    display(HTML(df.to_html(float_format=int_or_float_format_func,
                            classes=['alternate_colors3_groups'], escape=False)))
    
    
    # disattenuated correlations
    markdownstr = ['### Disattenuated correlations']
    
    markdownstr.append("The table shows correlations for {} machine scores for 'old' version and for {} scores for "
                      "'new' version".format(score_prefix_old, score_prefix_new))
    display(Markdown('\n'.join(markdownstr)))
    
    display(HTML(out_dfs['disattenuated_correlations'].to_html(index=False,
                                                               escape=False,
                                                               float_format=float_format_func,
                                                               classes=['alternate_colors2'])))
    
    for group in groups_eval:
        if not out_dfs['disattenuated_correlations_by_{}'.format(group)].empty:
            markdown_str = '#### Disattenuated correlations by {}'.format(group)
            display(Markdown(markdown_str))
            display(HTML(out_dfs['disattenuated_correlations_by_{}'.format(group)].to_html(index=True,
                                                                                       classes=['alternate_colors3_groups'],
                                                                                       float_format=int_or_float_format_func)))
